In [6]:
import os
import cv2
import numpy as np
import pandas as pd
import joblib
import tensorflow as tf
from sklearn.metrics import accuracy_score
from skimage.feature import hog

# Load the models
svm_model = joblib.load('rbf_svm_model.pkl')
dt_model = joblib.load('decision_tree_model.pkl')
rf_model = joblib.load('random_forest_model.pkl')

# Class names
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Load the test dataset
test_dir = 'D:/Ml2/Dataset/Testing'  # Replace with the actual path
IMG_HEIGHT = 180
IMG_WIDTH = 180

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=1,  # Load one image at a time
    shuffle=False  # Ensure that images are in the same order
)

# Function to extract HOG features
def extract_hog_features(image):
    image = (image * 255).astype(np.uint8)  # Ensure image is in the correct format
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features = hog(gray_image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), 
                   orientations=9, block_norm='L2-Hys', visualize=False)
    return features

# Prepare data for predictions
def prepare_test_data(dataset):
    features = []
    labels = []
    for images, lbl in dataset:
        img = images.numpy().astype('uint8')  # Convert tensor to numpy array
        feature = extract_hog_features(img[0])  # Extract HOG features for the first image
        features.append(feature)
        labels.append(lbl.numpy()[0])  # Append the true label
    return np.array(features), np.array(labels)

# Extract features from the test dataset
X_test, y_test = prepare_test_data(test_ds)

# Initialize lists to store predictions and probabilities
svm_probs = []
dt_probs = []
rf_probs = []

# Predict using all models and store probabilities
for i in range(len(X_test)):
    # SVM Prediction
    svm_pred = svm_model.predict([X_test[i]])[0]  # Predicted class
    if hasattr(svm_model, 'predict_proba'):  # Check if predict_proba is available
        svm_prob = svm_model.predict_proba([X_test[i]])[0]
        svm_probs.append(svm_prob)

    # Decision Tree Prediction
    dt_prob = dt_model.predict_proba([X_test[i]])[0]
    dt_pred = dt_model.predict([X_test[i]])[0]
    dt_probs.append(dt_prob)

    # Random Forest Prediction
    rf_prob = rf_model.predict_proba([X_test[i]])[0]
    rf_pred = rf_model.predict([X_test[i]])[0]
    rf_probs.append(rf_prob)

# Calculate average probabilities for each class
avg_svm_prob = np.mean(svm_probs, axis=0) if svm_probs else np.zeros(len(class_names))
avg_dt_prob = np.mean(dt_probs, axis=0)
avg_rf_prob = np.mean(rf_probs, axis=0)

# Calculate accuracy for each model
svm_accuracy = accuracy_score(y_test, [svm_pred for svm_pred in (svm_model.predict(X_test).tolist())])
dt_accuracy = accuracy_score(y_test, [np.argmax(p) for p in dt_probs])
rf_accuracy = accuracy_score(y_test, [np.argmax(p) for p in rf_probs])

# Create a DataFrame to summarize results
results_summary = pd.DataFrame({
    'Model': ['SVM', 'Decision Tree', 'Random Forest'],
    'Accuracy': [svm_accuracy, dt_accuracy, rf_accuracy],
    'Avg_Prob_Glioma': [avg_svm_prob[0], avg_dt_prob[0], avg_rf_prob[0]],
    'Avg_Prob_Meningioma': [avg_svm_prob[1], avg_dt_prob[1], avg_rf_prob[1]],
    'Avg_Prob_Notumor': [avg_svm_prob[2], avg_dt_prob[2], avg_rf_prob[2]],
    'Avg_Prob_Pituitary': [avg_svm_prob[3], avg_dt_prob[3], avg_rf_prob[3]],
})

# Display results summary DataFrame
print("\nResults Summary DataFrame:\n")
print(results_summary)


Found 1311 files belonging to 4 classes.

Results Summary DataFrame:

           Model  Accuracy  Avg_Prob_Glioma  Avg_Prob_Meningioma  \
0            SVM  0.911518         0.000000             0.000000   
1  Decision Tree  0.843631         0.198322             0.266209   
2  Random Forest  0.908467         0.209535             0.244081   

   Avg_Prob_Notumor  Avg_Prob_Pituitary  
0          0.000000            0.000000  
1          0.314264            0.221205  
2          0.318261            0.228124  


In [7]:
results_summary

,Model,Accuracy,Avg_Prob_Glioma,Avg_Prob_Meningioma,Avg_Prob_Notumor,Avg_Prob_Pituitary
0,SVM,0.911518,0.000000,0.000000,0.000000,0.000000
1,Decision Tree,0.843631,0.198322,0.266209,0.314264,0.221205
2,Random Forest,0.908467,0.209535,0.244081,0.318261,0.228124
